# Read Data

In [0]:
import os
from PIL import Image
import numpy as np

input_path = './result/'

letters = ['ე', 'ი', 'კ', 'ჟ', 'რ', 'ტ', 'ფ', 'ქ', 'ყ', 'ჰ']
ratio = 0.90
 
def getLetVec(letter):
    vec = np.zeros((10,), dtype=int)
    vec[letters.index(letter)] = 1
    return [vec]
 
def fill_data_for_folder(folder, train_data, test_data):
    files = os.listdir('{}/{}'.format(input_path,folder))
    result = getLetVec(folder)
    cur_data = []
    
    for file in files:
        image = Image.open("{}/{}/{}".format(input_path,folder, file))
        image = image.resize((40,40), resample = Image.BILINEAR)
        image_array = np.array(image)/255
        #print(image_array.shape)
        if image_array.shape == (40,40):
            #image_array = np.array(image)*(255.0/(np.array(image).max()))
            cur_data += [(image_array, result)]

    np.random.shuffle(cur_data)
    
    train_size = int(len(cur_data) * ratio)
    
    train_data += cur_data[:train_size]
    test_data += cur_data[train_size:]

def fill_data(train_data, test_data):
    folders = os.listdir(input_path)
    for folder in folders:
        print('Reading {}...'.format(folder))
        fill_data_for_folder(folder, train_data, test_data)
    print("Done Reading!")

if __name__ == "__main__":
    train_data = []
    test_data = []
    
    fill_data(train_data, test_data)
    train_data = np.array(train_data)
    test_data = np.array(test_data)

    np.random.shuffle(train_data)

    X_train = np.array(list(map(lambda x: x[0], train_data)))
    Y_train = np.array(list(map(lambda x: x[1], train_data)))
    
    X_test = np.array(list(map(lambda x: x[0], test_data)))
    Y_test = np.array(list(map(lambda x: x[1], test_data)))

    print("Total Train Data : {}".format(len(X_train)))
    print("Total Test  Data : {}".format(len(X_test)))

Reading ჟ...
Reading ტ...
Reading რ...
Reading ე...
Reading ქ...
Reading კ...
Reading ჰ...
Reading ყ...
Reading ფ...
Reading ი...
Done Reading!
Total Train Data : 10658
Total Test  Data : 1191


# Filter Class


In [0]:
import numpy as np

class Filter:
    
    ### constructor takes array which will be filter
    def __init__(self, filterM):
        self.filter = np.array(filterM)
        self.n = self.filter.shape[0]
        self.m = self.filter.shape[1]
        
    ### returns filter
    def get_filter(self):
        return self.filter
    
    ### change filter by delta matrix
    def add_delta(self, delta):
        self.filter += delta

    ### set the value of i,j element of the filter 
    def set(self, i, j, val):
        self.filter[i][j] = val

    ### returns the value of i,j element
    def get(self, i, j):
        return self.filter[i][j]

    ### takes matrix and returns filtered matrix
    def go_filter(self, data):
        resultdata = np.zeros((data.shape[0] - self.n + 1, data.shape[1] - self.m + 1))
        for i in range(data.shape[0] - self.n + 1):
            for j in range(data.shape[1] - self.m + 1):
                resultdata[i][j] = np.sum(self.filter * data[i:i+self.n,j:j+self.m])
        return resultdata
    
    ### takes matrix and poolfilter dimension, returns max-pooled matrix
    def go_maxpool(self, data, n, m):
        resultdata = np.zeros((int(data.shape[0] / n) , int(data.shape[1] / m)))
        for i in range(0, data.shape[0], n):
            for j in range(0, data.shape[1], m):
                resultdata[int(i/n)][int(j/m)] = np.max(data[i:i+n, j:j+m])
        return resultdata
    
    def go_maxpool(self,data,n,m):
        resultdata = np.zeros(data.shape[0] / n , data.shape[1] / m)
        for i in range(0, data.shape[0], n):
            for j in range(0,data.shape[1], m):
                resultdata[i/n][j/m] = data[i:i+n, j:j+m].mean()
        resultdata
        
        
    def avarage_pooling_backforward(self,data, loss_data, n, m):
        updated_data = np.zeros(data.shape)
        for i in range(0, data.shape[0], n):
            for j in range(0,data.shape[1], m):
                data[i: i + n, j: j + m] = (m*n)*resultdata[i/n][j/m]

    # pshiu, this method is pretty complex but in few words it updates convolutional filter
    # looking at loss matrix and pre-matrixes.
    def update_filter_one_matrix(self, data, loss_data):
        learning_rate = 0.0003
        updated_filter = np.zeros(self.filter.shape)
        updated_data = np.zeros(data.shape)
        for i in range(data.shape[0] + 1 - self.n):
            for j in range(data.shape[1] + 1 - self.m):
                # in summary resultdata[i][j] = self.filter * data[i:i+self.n, j : j+self.m]
                my_filter_part_data = data[i:i+self.n, j : j+self.m]
                my_data_part_data = updated_data[i:i+self.n, j : j+self.m]
                for x in range(my_filter_part_data.shape[0]):
                    for y in range(my_filter_part_data.shape[1]):
                        updated_filter[x][y] += loss_data[i][j]*my_filter_part_data[x][y]
                        # x derivative
                        my_data_part_data[x][y] +=loss_data[i][j]*self.filter[x][y]
                updated_data[i:i+self.n, j : j+self.m] = my_data_part_data
        self.filter =self.filter -  learning_rate*updated_filter
        return updated_filter, updated_data

# Layer Class

In [0]:

class Layer:
    ### constructor takes the list of filters 
    def __init__(self, num_filters):
        self.filters = []
        self.num_filters = num_filters
        self.matrixes = []
        self.preMatrixes = []
        self.n = 2
        self.m = 3
    
    ### returns the i-th filter
    def get_filter(self, i):
        return self.filters[i]
    
    ### return the list of filters
    def get_filters(self):
        return self.filters
    
    ### generates list of matrixes, from previous layer's matrixes and it's filters
    def gen_matrixes(self, preMatrixes):
        if len(self.filters) == 0:
            for x in range(len(preMatrixes)):
                each_filters = [Filter(np.random.randn(2,3)) for _ in range(self.num_filters)]
                self.filters.append(each_filters)
                
        self.preMatrixes = preMatrixes
        my_matrixes = [np.zeros((self.preMatrixes[0].shape[0] - self.n + 1, self.preMatrixes[0].shape[1] - self.m + 1)) for _ in range(self.num_filters)]

        for x in range(len(preMatrixes)):
            mat = preMatrixes[x]
            temp = 0
            for filt in self.filters[x]:
                my_matrixes[temp] += filt.go_filter(mat)
                temp+=1
        self.matrixes = my_matrixes
            
        
    ### returns i-th matrix of our matrixs' list
    def get_matrix(self, i):
        return self.matrix[i]
    
    ### returns list of matrixes
    def get_matrixes(self):
        return self.matrixes
    
    def layer_back_forward(self, loss_data_list):


        pre_matrix_list = self.preMatrixes
        updated_pre_matrix = [np.zeros(matr.shape) for matr in pre_matrix_list]
        
        updated_filters = []

        for x in range(len(self.filters)):
            each_filters = [np.zeros(matr.get_filter().shape) for matr in self.filters[x]]
            updated_filters.append(each_filters)
#         print(self.filters)
        for x in range(len(self.filters)):
            filt = self.filters[x]
            for index in range(len(filt)):
                cur_filt = filt[index]
                cur_loss_data = loss_data_list[index]
                for y in range(len(pre_matrix_list)):
                    mat = pre_matrix_list[y]
                    updated_filter, updated_data = cur_filt.update_filter_one_matrix(mat, cur_loss_data)
                    updated_filters[x][index]+=updated_filter
                    updated_pre_matrix[y]+=updated_data
        return updated_pre_matrix
           
    


# CNN Class

In [0]:
class CNN:
    ### constructor takes the starting matrix and filters list for each layer
    ### layerFilters is 4D list
    ### a = CNN(ar, )
    ## update.
    ## layerFilters listshi mewera listi filtrebisa layerebis mixedvit
    ## exla shignit ubralod ricxvs chavwer tu ramdeni filtri unda mas.
    def __init__(self, stMatrix, layerFilters):
        self.stMatrix = stMatrix
        self.layers = []
        for layerFilter in layerFilters:
#             print(layerFilter)
            self.layers.append(Layer(layerFilter))
    
    ### returns all layer objects 
    def get_layers(self):
        return self.layers
    def set_x(self,X):
        self.stMatrix = X
    
    ### returns i-th layer
    def get_layer(self, i):
        return self.layers[i]
        
    ### returns flattened np-array for fully connection NL
    def forward_prop(self):
        matrixList = [self.stMatrix]
        for layer in self.layers:
            layer.gen_matrixes(matrixList)
            matrixList = layer.get_matrixes()
        return self.flatten(matrixList)
            
    ### static method, flattens 3D array
    def flatten(self, matrixList):

        lst = []
        for matrix in matrixList:
            lst+=(matrix.flatten().tolist())
        return lst
    
    def unflatten(self, flatten_matrix):
        my_matrix = self.layers[len(self.layers)-1].get_matrixes()
        my_loss_matrix = [np.zeros(tmp.shape) for tmp in my_matrix]
        temp = 0
        for index in range(len(my_loss_matrix)):
            mat = my_loss_matrix[index]
            for x in range(mat.shape[0]):
                for y in range(mat.shape[1]):
                    my_loss_matrix[index][x][y] = flatten_matrix[0][temp]
                    temp += 1
        return my_loss_matrix
        
    def backward(self, delta_x):
        new_delta_x = delta_x
        for layer in reversed(self.layers):
            #WINEBIS listi da loss_datas listi
            new_delta_x = layer.layer_back_forward(new_delta_x)
    

# Dance NL Class

In [0]:
class Dense:
    
    ### constructor takes parameter's array(flattened array) and answer array(1x10)
    def __init__(self, X, Y):
        self.i=0
        self.X = X
        self.Y = Y
        self.cachedLoss = []
    
    # just return e^(-z)
    def my_exp(self,z):
        return np.exp(-z)
    
    def set_y(self, Y):
        self.Y=Y
    
    def set_matrix(self,mat):
        if self.X is None:
            self.W0 = np.random.randn(mat.size, 64)/100
            self.W1 = np.random.randn(64, 10)/100
        self.X = mat
        
        
        # our zigmoid function
    def zigmoid_function(self,z):
        return 1/(1 + self.my_exp(z))
    
    def costfn_class(self, X, y):
#         X = self.zigmoid_function(X)
        error= -1*y*np.log(X) + (y-1)*np.log(1-X)
#         my_sum = np.sum(abs(error))
        return error
    
    def cross_entropy(self, predictions, targets, epsilon=1e-10):
        predictions = np.clip(predictions, epsilon, 1. - epsilon)
        ce = targets*np.log(predictions+1e-9) + (targets - 1)*np.log(1-predictions + 1e-9)
        return ce
    
    
        # ferivative of our zigmoid function
    def derivative_zigmoid(self,z):
        return self.my_exp(z) / np.power((1 + self.my_exp(z)), 2)
    
    # our fucntion doing feed forward alghoritm.
    def feed_forward(self,X, W0, W1):
        h0 = np.dot(X, W0)  # our first matrix after using first hidden layer W0
    #     h0 = zigmoid_function(h0)  # our first matrix using zigmoid.
        
        h3 = np.dot(h0, W1) # our result after using second hidden layer W1
        h1 = self.zigmoid_function(h3) # our result matrix after using zigmoid.
        return h0, h1 # return result.
    
    # compute our error which is h1-y but we also take into consideration the zigmoid function.
    def my_error(self,h1):
#         error = h1 - self.Y
#         return error
        return self.cross_entropy(h1,self.Y)


    def backward(self,X, W0, W1, h1, h0, alfa_rate):
        # firstly lets compute error h1-y
        error = self.my_error(h1)
#         if self.cachedLoss != []:
#             error = error*0.1 + 0.9*self.cachedLoss
#         self.cachedLoss = error    
        
        bla_W1 = W1
        # first lets update W1 which is far easier W1 = (h0)T *(h1 - Y)
        transpose_h0 = np.transpose(h0)
        error = error * self.derivative_zigmoid(h1)

        # now lets update W0 too which is bit complex.
        # W0 = XT.(ERROR).W1T
        transpose_X = np.transpose(X)
        transpose_W1 = np.transpose(bla_W1)


        first_step = np.dot(error, transpose_W1)
        second_step = np.dot(transpose_X, first_step)
        delta_x = np.dot(error, np.transpose(np.dot(self.W0, self.W1)))
        W0 -= second_step*alfa_rate
        W1 -= np.dot (transpose_h0, error)*alfa_rate
        return delta_x
    

    def predict(self, back):
        h0, h1 = self.feed_forward(self.X, self.W0, self.W1)
        delta_x = None
        if back:
            delta_x = self.backward(self.X, self.W0, self.W1, h1, h0, 0.0005)
        return delta_x, h1

# **Initialization**

In [0]:


# first_layer_filters = []
# first_layer_filters.append(1)

# # here we will initialize second layer filters
# # second_layer_filter = Filter(np.random.randn(3,3))
# second_layer_filters = []
# second_layer_filters.append(1)

# here we will initialize second layer filters
filter_array = []
filter_array.append(3)
filter_array.append(3)

In [0]:
##### lets initialize CNN class
cnn = CNN(None,filter_array)
dense_class = Dense(None,None)

In [0]:
def predict(cnn,dense_class, back):
    flatten_array = np.array(cnn.forward_prop())
    flatten_array = np.reshape(np.array(flatten_array), (flatten_array.shape[0],1))
    
    #now lets start with dense class to work with neural network.
    flatten_array = np.transpose(flatten_array)
    
    # dense_class = Dense(np.transpose(flatten_array),Y)
    dense_class.set_matrix(flatten_array)
    delta_x, h1 = dense_class.predict(back)
    
    if back:
        #listi maq matricebis
        delta_x = cnn.unflatten(delta_x)
        cnn.backward(delta_x)
        
    return h1

In [0]:
def go_forest(X, Y, debug=False, back = True):
    cnn.set_x(X)
    dense_class.set_y(Y)
    for _ in range(1):
        h1 = predict(cnn, dense_class, back)
    return h1

In [0]:
def check_for_debug(index, total, percentage,):
    if percentage < 1:
        to_divide = total*percentage/100
    else:
        to_divide = int(total*percentage/100)
    if to_divide == 0 or index%to_divide == 0:
        return True
    return False
    
def print_train_debug(index, total, percentage):
    prc = (100*index/total)
    str = 'Done {}%!'.format(prc)
    print()
    print(str)
    
    bar = '=' * int(prc)
    bar += '-'*int(100-prc)
    bar_str = '[{}]'.format(bar)
    print(bar_str)
    
def print_test_debug(index, total, percentage, ans):
    prc = int(100*index/total)
    true_prc = 0
    if index:
        true_prc = int(100*ans/(index+1))
    str = 'Done {}%.. Result {}/{} ({}%)'.format(prc,ans,index,true_prc)
    print(str)
    print(index)

In [0]:
percentage = 1
# total = 100
total = int(len(X_train))
print(total)
ans = 0
ansbla = [0,0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0,0]
for index in range(0, total):
    debug = True
    if check_for_debug(index, total, percentage):
        print_test_debug(index, total, percentage, ans)
#         for i in range(10):
#             x = 0
#             if count[i] > 0:
#                 x = float(ansbla[i]/count[i])*100
#             print(letters[i],'----> ',x, '%')
        debug = True
    h = go_forest(X_train[index],Y_train[index], debug)
    ind1 = find_max_ind(h)
    ind2 = find_max_ind(Y_train[index]) 
    #count[ind2] += 1
    if ind1 == ind2:
        ans += 1
     #   ansbla[ind2] += 1
# print_train_debug(total, total, percentage)

In [0]:
def find_max_ind(arr):
    index = 0
    max_num = arr[0][0]
    for i in range(len(arr[0])):
        if arr[0][i] > max_num:
            max_num = arr[0][i]
            index = i
    return index

In [0]:
total = len(X_test)
percentage = 5
ans = 0
ansbla = [0,0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0,0]
for ind in range(total):
    if check_for_debug(ind, total, percentage):
        print_test_debug(ind, total, percentage, ans)
        for i in range(10):
            x = 0
            if count[i] > 0:
                x = float(ansbla[i]/count[i])*100
            print(letters[i],'----> ',x, '%')
    h = go_forest(X_test[ind],Y_test[ind], True, False)
    
    ind1 = find_max_ind(h)
    ind2 = find_max_ind(Y_test[ind])
    count[ind2] += 1
    if ind1 == ind2:
        ans += 1
        ansbla[ind2] += 1

In [0]:
len(X_test)

In [0]:

class Final:
    def __init__(self, cnnOBJ, denseOBJ):
        self.cnn = cnnOBJ
        self.dense = denseOBJ
        self.letters = ['ე', 'ი', 'კ', 'ჟ', 'რ', 'ტ', 'ფ', 'ქ', 'ყ', 'ჰ']
        
    def find_max_ind(arr):
      index = 0
      max_num = arr[0][0]
      for i in range(len(arr[0])):
          if arr[0][i] > max_num:
              max_num = arr[0][i]
              index = i
      return index
        
    def go(self, img):
        h = go_forest(img, [[1,0,0,0,0,0,0,0,0,0]], True, False)
        ind = find_max_ind(h)
        return self.letters[ind]
    
    def go_forest(X, Y, debug=False, back = True):
        self.cnn.set_x(X)
        self.dense.set_y(Y)
        for _ in range(1):
            h1 = predict(self.cnn, self.dense, back)
        return h1
    
    def predict(cnn, dense_class, back):
        flatten_array = np.array(cnn.forward_prop())
        flatten_array = np.reshape(np.array(flatten_array), (flatten_array.shape[0],1))

        #now lets start with dense class to work with neural network.
        flatten_array = np.transpose(flatten_array)

        # dense_class = Dense(np.transpose(flatten_array),Y)
        dense_class.set_matrix(flatten_array)
        delta_x, h1 = dense_class.predict(back)

        if back:
            #listi maq matricebis
            delta_x = cnn.unflatten(delta_x)
            cnn.backward(delta_x)

        return h1

FinalOBJ = Final(cnn,dense_class)    
    
import pickle

filehandler = open("savedModel1.pkl", 'wb') 
pickle.dump(FinalOBJ, filehandler)

In [0]:
!unzip './result.zip'